### Ingest lap_times folder

In [1]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

/home/ricardo/Documentos/Projects/dab-ci-cd/dab_project/.venv/lib/python3.12/site-packages/databricks/sdk/_widgets/__init__.py:71: UserWarning: 
To use databricks widgets interactively in your notebook, please install databricks sdk using:
	pip install 'databricks-sdk[notebook]'
Falling back to default_value_only implementation for databricks widgets.
  warnings.warn(


In [2]:
from formula1.formula1_constants import raw_folder_path, processed_folder_path, presentation_folder_path

In [3]:
from formula1.formula1_utils import add_ingestion_date, re_arrange_partition_column, df_column_to_list

##### Step 1 - Read the CSV file using the spark dataframe reader API

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [7]:
lap_times_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])

In [8]:
lap_times_df = spark.read \
.schema(lap_times_schema) \
.csv(f"{raw_folder_path}/lap_times")

##### Step 2 - Rename columns and add new columns
1. Rename driverId and raceId
1. Add ingestion_date with current timestamp

In [9]:
lap_times_with_ingestion_date_df = add_ingestion_date(lap_times_df)

In [10]:
from pyspark.sql.functions import lit

In [11]:
final_df = lap_times_with_ingestion_date_df.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumn("data_source", lit(v_data_source))

##### Step 3 - Write to output to processed container in parquet format

In [12]:
final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/lap_times")